In [21]:
!wget https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/phishtank_phishing_raw_reduced.csv
!wget https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/zenodo_benign_raw.csv

--2024-01-17 04:38:41--  https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/phishtank_phishing_raw_reduced.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3725551 (3.6M) [text/plain]
Saving to: ‘phishtank_phishing_raw_reduced.csv.1’

phishtank_phishing_ 100%[===================>]   3.55M  --.-KB/s    in 0.09s   

2024-01-17 04:38:41 (41.1 MB/s) - ‘phishtank_phishing_raw_reduced.csv.1’ saved [3725551/3725551]

--2024-01-17 04:38:41--  https://raw.githubusercontent.com/Topaz52/phishing-website-URL-dataset/main/zenodo_benign_raw.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.1

In [22]:
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import sequence
from prettytable import PrettyTable
import tensorflow as tf

In [23]:
# Some high level parameters:
show_top_n = 20
random_seed = 16

# Setting the random seed so that the code is repeatable.
np.random.seed(random_seed)

# **Load Dataset**

In [24]:
# Loading the current combined online-valid dataset.
# Use the dataset_downloader.py to download the current Phishtank online-valid.csv and update the combined dataset.
print("Phishtank Dataset")
phishtank_dataset = pd.read_csv("phishtank_phishing_raw_reduced.csv")
phishtank_dataset.tail(5)

# reduced dataset into 19,400
# save reduced 19,400 dataset to csv
# reduced_phistank_dataset = phishtank_dataset.sample(n=19400)
# reduced_phistank_dataset.to_csv('phishtank_phishing_raw_reduced.csv.csv', index= False)
# reduced_phistank_dataset.info()

Phishtank Dataset


,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
19395,8056354,https://kundendienst.builderallwppro.com/post/...,http://www.phishtank.com/phish_detail.php?phis...,2023-03-02T13:19:08+00:00,yes,2023-03-02T13:54:48+00:00,yes,Other
19396,7984714,https://v.ht/K0Si6?Home-Facebook-Confirmation=,http://www.phishtank.com/phish_detail.php?phis...,2022-12-22T22:36:02+00:00,yes,2022-12-22T22:42:12+00:00,yes,Facebook
19397,8066768,https://up02-02oapsza2.shopalo.co.pl/a1legro/l...,http://www.phishtank.com/phish_detail.php?phis...,2023-03-08T23:34:11+00:00,yes,2023-03-17T09:07:12+00:00,yes,Allegro
19398,8056037,http://www.kuerennkaycoato-co-jp.kuercnokaycca...,http://www.phishtank.com/phish_detail.php?phis...,2023-03-02T06:57:50+00:00,yes,2023-03-03T01:06:44+00:00,yes,Other
19399,7972750,http://1845651992185658.cf/,http://www.phishtank.com/phish_detail.php?phis...,2022-12-10T16:48:23+00:00,yes,2022-12-10T16:52:19+00:00,yes,Facebook


In [25]:
# Loading the whitelist from the 1 million most frequently visited domains.
print("Zenodo Benign Dataset")
zenodo_dataset = pd.read_csv("zenodo_benign_raw.csv")
zenodo_dataset.tail(5)

Zenodo Benign Dataset


,URLs,Label
19395,http://www.twitt.org/,Benign
19396,http://www.people.famouswhy.com/lucy_rangel/,Benign
19397,http://www.badoo.com/0139730059/,Benign
19398,http://www.mccrearyrecord.com/obituaries/x2018...,Benign
19399,http://www.daviswiki.org/Amtrak,Benign


# **Preprocessing**

> PhishTank

In [26]:
# Extracting tld and domain name.
tld_count = defaultdict(lambda: 0)
domain_names = []
for index, row in phishtank_dataset.iterrows():
    # Extracting the tld from the url
    domain_name = row["url"].replace("https://","").replace("http://","").replace("www.","").split("/")[0]
    domain_names.append(domain_name)
    tld = domain_name.split(".")[-1]
    tld_count[tld] += 1

tld_df = pd.Series(dict(tld_count))
tld_df.sort_values(ascending=False, inplace=True)
tld_print = tld_df.iloc[:show_top_n]
tld_print["OTHERS"] = tld_df.iloc[show_top_n:].sum()

In [27]:
# Adding the domain names extracted from the phishing urls as a new column.
phishtank_dataset["domain_names"] = domain_names

In [28]:
# drop column
phishtank_dataset_drop_df = phishtank_dataset.drop(['phish_id', 'phish_detail_url', 'verified', 'verification_time', 'online'],axis = 1)
phishtank_dataset_drop_df

,url,submission_time,target,domain_names
0,https://docs.google.com/presentation/d/e/2PACX...,2023-04-08T09:46:13+00:00,Other,docs.google.com
1,https://cajaitrujilllo.com/index.php,2023-03-28T18:41:15+00:00,Other,cajaitrujilllo.com
2,http://yxbho6gf0p.yennefer6664.pl,2023-03-14T09:43:02+00:00,Allegro,yxbho6gf0p.yennefer6664.pl
3,https://bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv...,2023-03-10T17:23:43+00:00,Other,bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv767gz74o...
4,https://caloundrawoodworkingclub.com.au/wp-adm...,2023-02-11T22:17:45+00:00,Other,caloundrawoodworkingclub.com.au
...,...,...,...,...
19395,https://kundendienst.builderallwppro.com/post/...,2023-03-02T13:19:08+00:00,Other,kundendienst.builderallwppro.com
19396,https://v.ht/K0Si6?Home-Facebook-Confirmation=,2022-12-22T22:36:02+00:00,Facebook,v.ht
19397,https://up02-02oapsza2.shopalo.co.pl/a1legro/l...,2023-03-08T23:34:11+00:00,Allegro,up02-02oapsza2.shopalo.co.pl
19398,http://www.kuerennkaycoato-co-jp.kuercnokaycca...,2023-03-02T06:57:50+00:00,Other,kuerennkaycoato-co-jp.kuercnokayccata.zxvtuo.top




> Zenodo



In [29]:
# Extracting tld and domain name.
tld_count = defaultdict(lambda: 0)
domain_names = []
for index, row in zenodo_dataset.iterrows():
    # Extracting the tld from the url
    domain_name = row["URLs"].replace("https://","").replace("http://","").replace("www.","").split("/")[0]
    domain_names.append(domain_name)
    tld = domain_name.split(".")[-1]
    tld_count[tld] += 1

tld_df = pd.Series(dict(tld_count))
tld_df.sort_values(ascending=False, inplace=True)
tld_print = tld_df.iloc[:show_top_n]
tld_print["OTHERS"] = tld_df.iloc[show_top_n:].sum()

In [30]:
# Adding the domain names extracted from the phishing urls as a new column.
zenodo_dataset["domain_names"] = domain_names

In [31]:
# Finding if there are any domains that are also in the whitelist.
domains_in_whitelist = np.intersect1d(phishtank_dataset_drop_df["domain_names"], zenodo_dataset["domain_names"])
# Tagging the whitelisted domains as such.
phishtank_dataset_drop_df["in_whitelist"] = np.in1d(phishtank_dataset_drop_df["domain_names"], domains_in_whitelist)

In [32]:
# Printing some data examples for reference.
print(phishtank_dataset_drop_df.shape[0], "rows")
print(tld_print.to_frame(name="TLD Count").transpose().to_string())
print(f"Percentage of top {show_top_n} tlds: {np.round(100*tld_df.iloc[:show_top_n].sum()/tld_df.sum(), decimals=2)} %")
phishtank_dataset_drop_df.head(20)

19400 rows
             com   org  net  edu   ca  gov  uk  info  au  us  eu  biz  tv  at  de  me  se  nl  ws  in  OTHERS
TLD Count  14524  2909  563  476  351  130  55    52  43  30  20   13  11  11  10   9   9   9   9   8     158
Percentage of top 20 tlds: 99.19 %


,url,submission_time,target,domain_names,in_whitelist
0,https://docs.google.com/presentation/d/e/2PACX...,2023-04-08T09:46:13+00:00,Other,docs.google.com,False
1,https://cajaitrujilllo.com/index.php,2023-03-28T18:41:15+00:00,Other,cajaitrujilllo.com,False
2,http://yxbho6gf0p.yennefer6664.pl,2023-03-14T09:43:02+00:00,Allegro,yxbho6gf0p.yennefer6664.pl,False
3,https://bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv...,2023-03-10T17:23:43+00:00,Other,bafybeieykrmvpnwd7ifrl4qjnxeaxfbic4vrv767gz74o...,False
4,https://caloundrawoodworkingclub.com.au/wp-adm...,2023-02-11T22:17:45+00:00,Other,caloundrawoodworkingclub.com.au,False
5,http://cooperadosicredi.com,2023-01-23T18:05:25+00:00,Other,cooperadosicredi.com,False
6,https://residencebidreams.it/rcb.nein/nnbs.php,2023-03-17T11:08:48+00:00,Other,residencebidreams.it,False
7,https://anmelden5telkom.de/auth,2023-03-29T02:41:24+00:00,Other,anmelden5telkom.de,False
8,http://pancakeswap.trading,2023-03-13T13:54:09+00:00,Other,pancakeswap.trading,False
9,https://connection-orangebank-certicode.derlma...,2023-02-22T15:54:12+00:00,Other,connection-orangebank-certicode.derlma.com,False


In [33]:
# Printing the top of the whitelist.
# print("Whitelist file:", zenodo_dataset)
# print(zenodo_dataset.shape[0], "rows")
print(zenodo_dataset.tail(20), "rows")

                                                    URLs   Label  \
19380       http://www.jeffersonrepublican.blogspot.com/  Benign   
19381    http://www.bruce-springsteen-blog.blogspot.com/  Benign   
19382                 http://www.sbsportspromotions.com/  Benign   
19383   http://www.talonbooks.com/authors/john-van-burek  Benign   
19384  http://www.issuu.com/k-statesports/docs/2009-1...  Benign   
19385  http://www.storiesofmyancestors.blogspot.com/2...  Benign   
19386             http://www.wn.com/Jacksonville_Jaguars  Benign   
19387           http://www.nybaseballdigest.com/?p=34885  Benign   
19388             http://www.robsilvermancounseling.com/  Benign   
19389       http://www.thecrittendenreport.blogspot.com/  Benign   
19390  http://www.dailyitem.com/0100_news/x1642542528...  Benign   
19391     http://www.knac.com/article.asp?ArticleID=8402  Benign   
19392  http://www.dailyadvocate.com/main.asp?SectionI...  Benign   
19393                     http://www.eastbayshut

In [34]:
# How many domains are in both the whitelist and in the phishing urls?
print()
print("Number of urls that have domains which are in the whilelist:", phishtank_dataset_drop_df["in_whitelist"].sum())

# Print the URLs that are in the whitelist
whitelisted_urls = phishtank_dataset_drop_df.loc[phishtank_dataset_drop_df["in_whitelist"], "url"]
print("Whitelisted URLs:")
for url in whitelisted_urls:
    print(url)


Number of urls that have domains which are in the whilelist: 97
Whitelisted URLs:
https://sites.google.com/view/upgrade-bt/home
https://sites.google.com/view/btlatestbill/home
https://sites.google.com/view/xmicrosoftoficew/home
https://sites.google.com/view/jmjmnhvdc/home
https://sites.google.com/amricalturs.net/playvoice/home
https://sites.google.com/view/fcg133/att
https://sites.google.com/view/office-v-ml-us/home
https://sites.google.com/view/plusforte/accueil
https://sites.google.com/view/clickpagenewlogin2021
https://sites.google.com/view/klicken-sie-hierumsich-beig/home
https://sites.google.com/view/serv-appli-ob-fr-2022/accueil
https://sites.google.com/view/onlineemail890/home?authuser=1
https://sites.google.com/view/manager3-controlpanel-ovh
https://sites.google.com/view/newbtmissedcall/home
https://sites.google.com/view/htvvss/home?authuser=3
https://sites.google.com/view/yahoonice/yahoo-com
https://sites.google.com/view/presenter1
https://sites.google.com/view/noticeplaypage

In [35]:
# For the dataset, excluding all where the domain name is in the whitelist.
# Since if the domain is in both lists we cannot tell if its safe or phishing? --> This helps with labeleling the data.
phishtank_dataset_drop_df_without_intersection = phishtank_dataset_drop_df.loc[phishtank_dataset_drop_df['in_whitelist'] == False]
zenodo_dataset_df_without_intersection = zenodo_dataset.loc[np.invert(zenodo_dataset['domain_names'].isin(domains_in_whitelist))]

In [39]:
phishtank_dataset_drop_df_without_intersection_reduced = phishtank_dataset_drop_df_without_intersection.sample(n=19300)
zenodo_dataset_df_without_intersection_reduced = zenodo_dataset_df_without_intersection.sample(n=19300)

In [40]:
# Storing the preprocessed phishtank and zenodo URLs fatures to csv file
phishtank_dataset_drop_df_without_intersection_reduced.to_csv('cleaned_rnn_phishtank_sampled.csv', index= False)
zenodo_dataset_df_without_intersection_reduced.to_csv('cleaned_rnn_zenodo_sampled.csv', index= False)